In [2]:
#import necessary libraries
import re
import gensim
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

ModuleNotFoundError: No module named 'gensim'

In [3]:
#reading file
file = open("mayowa.txt","r") 
data=file.readlines() 
file.close()

In [3]:

def preprocessor(text):
    """
    
    Lowers the case of the input, removes everything inside [] then removes 's and fetches only ascii characters

    :param text: blah
    :type text: string
    :returns: tokens-> blah
    :rtype: string

    """
    newString = text.lower()
    newString = re.sub("[\(\[].*?[\)\]]", "", newString)
    newString = re.sub("'s","",newString)
    newString = re.sub("[^'0-9.a-zA-Z]", " ", newString)
    tokens=newString.split()
    return (" ".join(tokens)).strip()

"str(object='') -> str\nstr(bytes_or_buffer[, encoding[, errors]]) -> str\n\nCreate a new string object from the given object. If encoding or\nerrors is specified, then the object must expose a data buffer\nthat will be decoded using the given encoding and error handler.\nOtherwise, returns the result of object.__str__() (if defined)\nor repr(object).\nencoding defaults to sys.getdefaultencoding().\nerrors defaults to 'strict'."

In [5]:
#call above function
text=[]
for i in data:
    text.append(preprocessor(i))

all_sentences=[]    
for i in text:
    sentences=i.split(".")       
    for i in sentences:
        if(i!=''):
            all_sentences.append(i.strip())

In [6]:
# tokenizing the sentences for training word2vec
tokenized_text = [] 
for i in all_sentences:
    tokenized_text.append(i.split()) 

In [7]:
#define word2vec model
model_w2v = gensim.models.Word2Vec(
            tokenized_text,
            size=200, # desired no. of features/independent variables 
            window=5, # context window size
            min_count=2,
            sg = 0, # 1 for cbow model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 2, # no.of cores
            seed = 34)

In [8]:
#train word2vec
model_w2v.train(tokenized_text, total_examples= len(tokenized_text), epochs=model_w2v.epochs)


(19002, 35240)

In [9]:

def word_vector(tokens, size):
    """

    This function obtains the sentence embedding
    
    :param tokens: blah
    :type tokens: string
    :param size: blah
    :type size: integer
    :returns: vec->blah
    :rtype: vector

    """
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not in vocabulary
                         
            continue
    if count != 0:
        vec /= count
    return vec

In [10]:
#call above function
wordvec_arrays = np.zeros((len(tokenized_text), 200))
for i in range(len(tokenized_text)):
    wordvec_arrays[i,:] = word_vector(tokenized_text[i], 200)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [11]:
# similarity matrix
sim_mat = np.zeros([len(wordvec_arrays), len(wordvec_arrays)])


In [12]:
#compute similarity score
for i in range(len(wordvec_arrays)):
  for j in range(len(wordvec_arrays)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(wordvec_arrays[i].reshape(1,200), wordvec_arrays[j].reshape(1,200))[0,0]


In [13]:
#Generate a graph
nx_graph = nx.from_numpy_array(sim_mat)

In [14]:
#compute pagerank scores
scores = nx.pagerank(nx_graph)

In [16]:
#sort the scores
sorted_x = sorted(scores.items(), key=lambda kv: kv[1],reverse=True)

sent_list=[]
for i in sorted_x:
    sent_list.append(i[0])


In [17]:
#extract top 10 sentences
num=10
summary=''
for i in range(num):
    summary=summary+all_sentences[sent_list[i]]+'. '
print(summary)

the highlights for him included hitting a hundred with a swollen jaw and helping india avoid the follow on in the process at georgetown in the drawn first test contributing with a fifty and four catches to india victory in the second test at port of spain india first test victory in west indies since 1975 76 and another fifty in the drawn fourth test with a wicket to boot that of ridley jacobs who was batting on 118. dravid who had been knocking at the doors of indian national cricket team for quite a while with his consistent performance in domestic cricket received his first national call in october 1994 for the last two matches of the wills world series. when dravid joined laxman in the middle on the third day of the test with scoreboard reading 232 4 and india still needing 42 runs to avoid an innings defeat another convincing win for australia looked inevitable. india failed to qualify for the semi finals having lost to australia and new zealand but achieved a consolation victory 